In [1]:
import pandas as pd

def combine_csv_files(file1, file2, output_file=None):
    """
    Combines two CSV files with identical columns into a single DataFrame.

    Parameters:
    - file1 (str): Path to the first CSV file
    - file2 (str): Path to the second CSV file
    - output_file (str, optional): Path to save the combined CSV. If None, no file is saved.

    Returns:
    - pd.DataFrame: Combined DataFrame containing all rows from both files

    Raises:
    - ValueError: If the columns in the two files don't match
    - FileNotFoundError: If either file cannot be found
    """
    try:
        # Read the two CSV files
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)

        # Check if columns match
        if set(df1.columns) != set(df2.columns):
            raise ValueError("The CSV files have different columns. They must have identical columns to be combined.")

        # Combine the DataFrames
        combined_df = pd.concat([df1, df2], ignore_index=True)

        # Optionally save to a new CSV file
        if output_file:
            combined_df.to_csv(output_file, index=False)
            print(f"Combined data saved to '{output_file}'")

        print(f"Combined dataset contains {len(combined_df)} rows")
        return combined_df

    except FileNotFoundError as e:
        print(f"Error: One or both files not found - {e}")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

# Example usage
if __name__ == "__main__":
    # Example filenames (replace with your actual file paths)
    file1 = "bus_status_dataset1.csv"
    file2 = "bus_status_dataset.csv"
    output = "combined_bus_status.csv"

    # Combine the files and save the result
    combined_data = combine_csv_files(file1, file2, output)

    # Display basic info about the combined data
    print("\nColumns in combined dataset:")
    print(combined_data.columns.tolist())
    print("\nFirst few rows:")
    print(combined_data.head())

Combined data saved to 'combined_bus_status.csv'
Combined dataset contains 163 rows

Columns in combined dataset:
['bus_id', 'trip_id', 'route_id', 'current_lat', 'current_lon', 'next_stop_id', 'next_stop_lat', 'next_stop_lon', 'next_stop_name', 'current_time', 'position_timestamp', 'expected_arrival_time', 'time_to_arrival_seconds', 'distance_to_stop_meters', 'speed_m_s', 'status', 'stop_sequence', 'wheelchair_boarding']

First few rows:
   bus_id                           trip_id  route_id  current_lat  \
0    1010  5341__201065_Timetable_-_2025-02       211    43.854668   
1    6100  1475__201086_Timetable_-_2025-02       900    43.884548   
2    6101  3506__201038_Timetable_-_2025-02       302    43.946533   
3    6102  3003__201072_Timetable_-_2025-02       101    43.820915   
4    6103  1555__401094_Timetable_-_2025-02       900    43.857716   

   current_lon  next_stop_id  next_stop_lat  next_stop_lon  \
0   -79.055603          1601      43.853329     -79.060678   
1   -78.9219

In [5]:
import pytz
from datetime import datetime

def convert_to_unix_timestamp(csv_file, output_file):
    """
    Converts Eastern Time timestamps in format '2025-03-22T15:54:50' to Unix timestamps.

    Parameters:
    - csv_file (str): Path to the input CSV file
    - output_file (str): Path to save the modified CSV file
    """
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Define Eastern Time timezone
    eastern = pytz.timezone('America/New_York')

    # Function to convert a single timestamp to Unix time
    def to_unix(timestamp):
        try:
            # Parse the timestamp string and localize to Eastern Time
            dt = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S')
            dt = eastern.localize(dt)
            # Convert to UTC and then to Unix timestamp
            dt_utc = dt.astimezone(pytz.UTC)
            return int(dt_utc.timestamp())
        except (ValueError, TypeError, AttributeError):
            return timestamp  # Return original value if conversion fails

    # Identify columns that might contain timestamps
    time_columns = []
    for col in df.columns:
        # Check if any value in the column matches the expected format
        sample = df[col].dropna().iloc[0] if not df[col].dropna().empty else None
        if sample and isinstance(sample, str) and len(sample) == 19 and 'T' in sample:
            try:
                datetime.strptime(sample, '%Y-%m-%dT%H:%M:%S')
                time_columns.append(col)
            except ValueError:
                continue

    # Convert identified timestamp columns to Unix time
    for col in time_columns:
        df[col] = df[col].apply(to_unix)

    # Save the modified DataFrame to a new CSV
    df.to_csv(output_file, index=False)

    print(f"Converted timestamps in columns: {time_columns}")
    print(f"Modified dataset saved to '{output_file}'")
    print(f"Number of rows: {len(df)}")
    print("\nFirst few rows of the modified dataset:")
    print(df.head())

# Example usage
if __name__ == "__main__":
    input_file = 'bus_status_dataset.csv'  # Replace with your CSV filename
    output_file = 'bus_status_dataset_unix.csv'

    convert_to_unix_timestamp(input_file, output_file)

Converted timestamps in columns: ['current_time', 'position_timestamp', 'expected_arrival_time']
Modified dataset saved to 'bus_status_dataset_unix.csv'
Number of rows: 2146

First few rows of the modified dataset:
   bus_id                           trip_id  route_id  current_lat  \
0    1013  1607__261023_Timetable_-_2025-02       302    43.880016   
1    6101  2940__261012_Timetable_-_2025-02       915    43.854618   
2    6103   500__461021_Timetable_-_2025-02       403    43.886902   
3    6104  1606__261025_Timetable_-_2025-02       905    43.903885   
4    6106   563__461016_Timetable_-_2025-02       902    43.904518   

   current_lon  next_stop_id  next_stop_lat  next_stop_lon  \
0   -78.942314           208      43.884855     -78.944315   
1   -79.041199          2282      43.853799     -79.041066   
2   -78.877434          1287      43.886764     -78.876312   
3   -78.920181           281      43.899509     -78.918233   
4   -78.833870          1136      43.905928     -78.82